In [1]:
from run_bench import save_overall_results, dataset_metadata
from pathlib import Path
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import dash

/Users/user/demo_1/oobabooga_macos/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fig = save_overall_results(results_folder=Path("results"), method="plotly")
fig

100%|██████████| 9/9 [00:00<00:00, 34600.12it/s]


Loading datasets with seed: 1
0.013548612594604492


100%|██████████| 9/9 [00:00<00:00, 101203.05it/s]


Loading datasets with seed: 1
0.009061813354492188


100%|██████████| 9/9 [00:00<00:00, 95808.97it/s]


...

Loading datasets with seed: 1
0.011076211929321289


100%|██████████| 9/9 [00:00<00:00, 96791.63it/s]


Loading datasets with seed: 1
0.011234045028686523


100%|██████████| 9/9 [00:00<00:00, 108162.57it/s]


Loading datasets with seed: 1
0.008485794067382812


In [3]:
results_df = pl.read_parquet(Path("results/results.parquet"))
# sort by score
results_df = results_df.sort("score").reverse()
# extract score and model columns
# max model col width
with pl.Config(fmt_str_lengths=40, tbl_rows=1000):
    print(results_df.select(["model", "normalized_score", "score"]))

shape: (17, 3)
┌───────────────────────────────────────────┬──────────────────┬───────┐
│ model                                     ┆ normalized_score ┆ score │
│ ---                                       ┆ ---              ┆ ---   │
│ str                                       ┆ f64              ┆ i64   │
╞═══════════════════════════════════════════╪══════════════════╪═══════╡
│ dolphin-2.0-mistral-7b.Q6_K.gguf          ┆ 0.562778         ┆ 1013  │
│ speechless-llama2-hermes-orca-platypus-w… ┆ 0.558333         ┆ 1005  │
│ superplatty-30b.Q2_K.gguf                 ┆ 0.556111         ┆ 1001  │
│ mythical-destroyer-v2-l2-13b.Q6_K.gguf    ┆ 0.552778         ┆ 995   │
│ Mistral-11B-CC-Air.q6_k.gguf              ┆ 0.534444         ┆ 962   │
│ athena-v4.Q6_K.gguf                       ┆ 0.532222         ┆ 958   │
│ zephyr-7b-alpha.Q6_K.gguf                 ┆ 0.526111         ┆ 947   │
│ Emerhyst-20B.q6_k.gguf                    ┆ 0.514444         ┆ 926   │
│ xwin-lm-13b-v0.1.Q6_K.gguf        

In [4]:
row_dicts = results_df.select(["model", "normalized_score", "normalized_category_scores"]).to_dicts()
for row in row_dicts:
    for d in sorted(list(row["normalized_category_scores"])):
        row[d] = row["normalized_category_scores"].get(d)
    del row["normalized_category_scores"]
scores_df = pl.DataFrame(row_dicts)
scores_df


model,normalized_score,Comprehension/Reading Comprehension,Knowledge/Multi-subject Test,Math/Mathmatical Reasoning,Reasoning/Commonsense Reasoning,Safety/Truthfulness
str,f64,f64,f64,f64,f64,f64
"""dolphin-2.0-mi…",0.562778,0.74,0.665,0.19,0.5725,0.515
"""speechless-lla…",0.558333,0.775,0.6675,0.255,0.5475,0.47
"""superplatty-30…",0.556111,0.85,0.6475,0.25,0.515,0.55
"""mythical-destr…",0.552778,0.76,0.6425,0.245,0.56,0.445
"""Mistral-11B-CC…",0.534444,0.705,0.6575,0.215,0.5175,0.505
"""athena-v4.Q6_K…",0.532222,0.685,0.62,0.285,0.52625,0.475
"""zephyr-7b-alph…",0.526111,0.675,0.67,0.23,0.505,0.47
"""Emerhyst-20B.q…",0.514444,0.725,0.5975,0.25,0.495,0.48
"""xwin-lm-13b-v0…",0.487778,0.69,0.5675,0.24,0.50125,0.32


In [5]:
row_dicts = results_df.select(["model", "normalized_score", "score", "normalized_scores"]).to_dicts()
for row in row_dicts:
    for d in sorted(list(row["normalized_scores"])):
        row[d] = row["normalized_scores"].get(d)
    del row["normalized_scores"]
scores_df = pl.DataFrame(row_dicts)
scores_df

model,normalized_score,score,ai2_arc-arc-challenge-validation,alekseykorshuk_hellaswag-arc-easy-validation,anli--test_r1,cais_mmlu-all-validation,cosmos_qa--validation,math_qa--validation,race-high-validation,truthful_qa-multiple_choice-validation,winogrande-winogrande_s-validation
str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""dolphin-2.0-mi…",0.562778,1013,0.83,0.695,0.62,0.5,0.77,0.19,0.74,0.515,0.205
"""speechless-lla…",0.558333,1005,0.78,0.65,0.615,0.555,0.77,0.255,0.775,0.47,0.155
"""superplatty-30…",0.556111,1001,0.81,0.525,0.485,0.485,0.77,0.25,0.85,0.55,0.28
"""mythical-destr…",0.552778,995,0.79,0.665,0.66,0.495,0.765,0.245,0.76,0.445,0.15
"""Mistral-11B-CC…",0.534444,962,0.78,0.455,0.42,0.535,0.745,0.215,0.705,0.505,0.45
"""athena-v4.Q6_K…",0.532222,958,0.755,0.61,0.5,0.485,0.715,0.285,0.685,0.475,0.28
"""zephyr-7b-alph…",0.526111,947,0.785,0.56,0.465,0.555,0.75,0.23,0.675,0.47,0.245
"""Emerhyst-20B.q…",0.514444,926,0.715,0.565,0.44,0.48,0.725,0.25,0.725,0.48,0.25
"""xwin-lm-13b-v0…",0.487778,878,0.63,0.46,0.41,0.505,0.67,0.24,0.69,0.32,0.465


In [6]:
# from transformers import AutoTokenizer
# from petals import AutoDistributedModelForCausalLM

# # Choose any model available at https://health.petals.dev
# model_name = "petals-team/StableBeluga2"  # This one is fine-tuned Llama 2 (70B)

# # Connect to a distributed network hosting model layers
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
# prompt = """
# There are three sisters in a room alone. Anna is reading a book. Alice is playing a game called Zonda. Zonda requires two people to play it.
# What is the third sister, Amanda, most likely doing? Explain why.
# Select from the following options:
# (A) Also reading a book, like Anna
# (B) Having music lessons from Alice
# (C) Playing Zonda with Alice
# (D) Observing the other sisters, while they do their activities only
# (E) Trying to think of something to so
# Answer: (
# """
# from time import time
# # Run the model as if it were on your computer
# inputs = tokenizer(prompt, return_tensors="pt")["input_ids"]
# s = time()
# outputs = model.generate(inputs, max_new_tokens=100)
# print(tokenizer.decode(outputs[0]))
# print(f"Time taken: {time() - s:.2f}s")

In [8]:
from pathlib import Path
import json
from run_bench import get_datasets, calculate_category_scores

results_folder = Path("results")

results_df = pl.DataFrame()
results_folders = [f for f in Path(results_folder).iterdir() if not f.is_file()]
for result_folder in results_folders:
    print(result_folder)
    metadata = json.load(
        open(result_folder / "metadata.json", "r"),
    )
    datasets_df = pl.read_parquet(
        result_folder / "datasets_results.parquet",
    )
    number_of_samples = metadata["number_of_samples"]

    datasets = get_datasets(number_of_samples, metadata["dataset_seed"])

    scores = {
        r["dataset"]: r["score"]
        for r in datasets_df.group_by("dataset")
        .agg(pl.sum("score").alias("score"))
        .to_dicts()
    }
    normalized_scores = {k: v / number_of_samples for k, v in scores.items()}
    category_scores = calculate_category_scores(datasets_df)
    normalized_category_scores = {
        k: v / (len([d for d in datasets if d.category == k]) * number_of_samples)
        for k, v in category_scores.items()
    }
    score = sum(scores.values())
    normalized_score = score / (len(metadata["datasets"]) * number_of_samples)
    # normalized_score = sum(normalized_category_scores.values())/len(normalized_category_scores)

    result = {
        "model": metadata["model"],
        "model_seed": metadata["model_seed"],
        "dataset_seed": metadata["dataset_seed"],
        "score": score,
        "normalized_score": normalized_score,
        "scores": scores,
        "normalized_scores": normalized_scores,
        "category_scores": category_scores,
        "normalized_category_scores": normalized_category_scores,
        "number_of_samples": number_of_samples,
        "run_folder_name": result_folder.name,
    }
    results_df = pl.DataFrame([*results_df.to_dicts(), result])
results_df.sort("normalized_score").reverse().select(["model", "normalized_score", "score"])

results/tinyllama-1.1b-chat-v0.3.Q6_K.gguf-ms-1-ds-1-2023-10-10T17-36-37-252433


100%|██████████| 9/9 [00:00<00:00, 83330.54it/s]


Loading datasets with seed: 1
0.019944190979003906
results/xwin-lm-7b-v0.1.Q6_K.gguf-ms-1-ds-1-2023-10-11T16-07-37-341887


100%|██████████| 9/9 [00:00<00:00, 100129.27it/s]


Loading datasets with seed: 1
0.00999593734741211
results/Mistral-11B-CC-Air.q6_k.gguf-ms-1-ds-1-2023-10-11T20-00-57-110729


...

100%|██████████| 9/9 [00:00<00:00, 97541.95it/s]


Loading datasets with seed: 1
0.009710073471069336
results/dolphin-2.0-mistral-7b.Q6_K.gguf-ms-1-ds-1-2023-10-11T14-38-22-445488


100%|██████████| 9/9 [00:00<00:00, 86579.67it/s]


Loading datasets with seed: 1
0.04532814025878906


model,normalized_score,score
str,f64,i64
"""dolphin-2.0-mi…",0.562778,1013
"""speechless-lla…",0.558333,1005
"""superplatty-30…",0.556111,1001
"""mythical-destr…",0.552778,995
"""Mistral-11B-CC…",0.534444,962
"""athena-v4.Q6_K…",0.532222,958
"""zephyr-7b-alph…",0.526111,947
"""Emerhyst-20B.q…",0.514444,926
"""xwin-lm-13b-v0…",0.487778,878
